## Цели и план работы

 
* ознакомление с предоставленными датасетами и описаниями представленных данных;
* оценка чистоты и полноты данных;  
* приведение данных в удобный вид для дальнейшей работы;
* проведение базовой чистки (дубликаты, пустые значения, типизация данных, ненужные атрибуты);
* анализ ключевых распределений данных и их взаимосвязей. 

**Цель проекта**: разработать модель предсказания совершения одного из целевых действий ("Заказать звонок", "Оставить заявку") для сессий по введенным атрибутам типа utm_*, device_*, geo_* и упаковать модель в сервис.

![title](data/Цель.png)

![title](data/Формат.png)

## *Импорт библиотек и модулей*

In [3]:
import pandas as pd
import json
import gc
import pickle

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV

## *Загрузка данных*

In [142]:
df_hits = pd.read_csv('data/ga_hits.csv')
print(f'Размеры датасета: {df_hits.shape}')

Размеры датасета: (15726470, 11)


In [143]:
df_session = pd.read_csv('data/ga_sessions.csv', low_memory=False)
print(f'Размеры датасета: {df_session.shape}')

Размеры датасета: (1860042, 18)


## *Анализ и описание датасетов*

In [4]:
df_hits

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15726465,6866159858916559617.1640270865.1640270865,2021-12-23,810589.0,43,event,NaN,sberauto.com/cars/all/toyota/fortuner/24cb5af2...,quiz,quiz_show,NaN,NaN
15726466,7310304587364460692.1640261783.1640261783,2021-12-23,904927.0,40,event,NaN,sberauto.com/cars/all/mercedes-benz/gla-klasse...,quiz,quiz_show,NaN,NaN
15726467,8013702685784312179.1640270195.1640270195,2021-12-23,2172865.0,43,event,NaN,sberauto.com/cars/all/toyota/alphard/2ebe4871?...,quiz,quiz_show,NaN,NaN
15726468,8021505554734405918.1640257821.1640257821,2021-12-23,713325.0,45,event,NaN,sberauto.com/cars/all/bmw/x3/6a660f0a?rental_p...,quiz,quiz_show,NaN,NaN


В датасете `ga_hits.csv` содержатся данные о событиях в рамках каждой сессии из другого датасета `ga_sessions.csv`

![title](data/ga_hits1.png)

In [5]:
df_session

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1860037,9055415581448263752.1640159305.1640159305,2108378238.1640159304,2021-12-22,10:48:25,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,VlqBmecIOXWjCWUmQkLd,desktop,Windows,NaN,NaN,1920x1080,Chrome,Russia,Moscow
1860038,9055421130527858185.1622007305.1622007305,2108379530.1622007305,2021-05-26,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,390x844,Safari,Russia,Stavropol
1860039,9055422955903931195.1636979515.1636979515,2108379955.1636979515,2021-11-15,15:31:55,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1860040,905543020766873816.1638189404.1638189404,210838164.1638189272,2021-11-29,15:36:44,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x851,Chrome,Russia,Chelyabinsk


Описание атрибутов `ga_sessions.csv`

![title](data/ga_sessions.png)

Целевые действия находятся в колонке `event_action`.

Целевое действие — события типа «Оставить заявку» и «Заказать звонок»
(ga_hits.event_action in ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']).

Целевое действие считается выполненым, если для сессии из `sessions` есть хотя бы одно целевое событие в `hits`. 

## *Data Preparation*

переменная с целевыми действиями

In [144]:
# переменная с целевыми действиями
all_targets = ['sub_car_claim_click',

'sub_car_claim_submit_click',

'sub_open_dialog_click',

'sub_custom_question_submit_click',

'sub_call_number_click',

'sub_callback_submit_click',

'sub_submit_success',

'sub_car_request_submit_click']

проверим наличие пропусков в колонках `session_id` и `event_action` в датасете `df_hits`

In [145]:
# проверим наличие пропусков в колонках 'session_id' и 'event_action'
df_hits[['session_id', 'event_action']].isna().sum().sort_values(ascending=False)

session_id      0
event_action    0
dtype: int64

проверим наличие пропусков в датасете `df_session`

In [146]:
# проверим наличие пропусков в датасете `df_session`
df_session.isna().sum().sort_values(ascending=False)

device_model                1843704
utm_keyword                 1082061
device_os                   1070138
device_brand                 367178
utm_adcontent                335615
utm_campaign                 219603
utm_source                       97
geo_country                       0
device_browser                    0
device_screen_resolution          0
session_id                        0
device_category                   0
client_id                         0
utm_medium                        0
visit_number                      0
visit_time                        0
visit_date                        0
geo_city                          0
dtype: int64

In [147]:
# заполняем столбец df_hits['target'] "1", если df_ga_hits['event_action'] содержит целевую, иначе "0"
df_hits['target'] = df_hits['event_action'].apply(lambda x: 1 if x in all_targets else 0)
df_hits['target'].value_counts()

target
0    15621562
1      104908
Name: count, dtype: int64

распределение целого действия:

*Всего целевых действий 104908 из 15726470, т.е процент не велик, т.к. продукт новый и не дешёвый*

Дополним датасет `df_session` целевыми переменными из столбеца `target` датасета `df_hits`

In [148]:
# группируем по 'session_id' и по столбцу 'target'
target = df_hits.groupby('session_id')[['target']].max().reset_index()
target

,session_id,target
0,1000009318903347362.1632663668.1632663668,0
1,1000010177899156286.1635013443.1635013443,0
2,1000013386240115915.1635402956.1635402956,0
3,1000017303238376207.1623489300.1623489300,0
4,1000020580299877109.1624943350.1624943350,0
...,...,...
1734605,999960188766601545.1626816843.1626816843,0
1734606,99996598443387715.1626811203.1626811203,0
1734607,999966717128502952.1638428330.1638428330,0
1734608,999988617151873171.1623556243.1623556243,0


In [149]:
# добавляем к df_session столбец target
df_session_target = pd.merge(target, df_session)
df_session_target.shape

(1732266, 19)

In [150]:
df_session_target

,session_id,target,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,1000009318903347362.1632663668.1632663668,0,232832813.1632663714,2021-09-26,16:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,Samsung,NaN,412x869,Chrome,Russia,Gelendzhik
1,1000010177899156286.1635013443.1635013443,0,232833013.1635013438,2021-10-23,21:24:03,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,360x640,Samsung Internet,Russia,Voronezh
2,1000013386240115915.1635402956.1635402956,0,232833760.1635402955,2021-10-28,09:35:56,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,412x846,Chrome,Russia,Cherkessk
3,1000017303238376207.1623489300.1623489300,0,232834672.1623489295,2021-06-12,12:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Realme,NaN,360x780,Chrome,Russia,Irkutsk
4,1000020580299877109.1624943350.1624943350,0,232835435.1624943349,2021-06-29,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x736,Safari,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,999960188766601545.1626816843.1626816843,0,232821374.1626816841,2021-07-21,00:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,NaN,mobile,NaN,Huawei,NaN,360x780,Chrome,Russia,Moscow
1732262,99996598443387715.1626811203.1626811203,0,23282272.1626811203,2021-07-20,23:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,375x667,Safari,Russia,Saint Petersburg
1732263,999966717128502952.1638428330.1638428330,0,232822894.1638428328,2021-12-02,09:58:50,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x873,Chrome,Russia,Nizhny Novgorod
1732264,999988617151873171.1623556243.1623556243,0,232827993.1623556243,2021-06-13,06:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,412x732,Samsung Internet,Russia,Chelyabinsk


In [151]:
df_session_target["target"].value_counts(dropna=False, normalize=True).apply(lambda x: f'{x:0.2%}')

target
0    97.10%
1     2.90%
Name: proportion, dtype: object

2.9% всех сессий закончилось совершением целевого действия

In [15]:
collected = gc.collect()
print("«Сборщик мусора: собрано»",
          "%d objects." % collected)

«Сборщик мусора: собрано» 40 objects.


удалим не используемые датафреймы, чтобы не занимал много памяти

In [16]:
del df_hits
del df_session
del target

collected = gc.collect()
print("«Сборщик мусора: собрано»",
          "%d objects." % collected)

«Сборщик мусора: собрано» 0 objects.


In [18]:
# запишем в файл csv соединенный датасет
df_session_target.to_csv('data/df_session_target.csv', index=False)
# загрузим подготовленный df_session2
df = pd.read_csv('data/df_session_target.csv', low_memory=False)

In [152]:
df_session_target

,session_id,target,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,1000009318903347362.1632663668.1632663668,0,232832813.1632663714,2021-09-26,16:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,NaN,mobile,NaN,Samsung,NaN,412x869,Chrome,Russia,Gelendzhik
1,1000010177899156286.1635013443.1635013443,0,232833013.1635013438,2021-10-23,21:24:03,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,360x640,Samsung Internet,Russia,Voronezh
2,1000013386240115915.1635402956.1635402956,0,232833760.1635402955,2021-10-28,09:35:56,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,NaN,412x846,Chrome,Russia,Cherkessk
3,1000017303238376207.1623489300.1623489300,0,232834672.1623489295,2021-06-12,12:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Realme,NaN,360x780,Chrome,Russia,Irkutsk
4,1000020580299877109.1624943350.1624943350,0,232835435.1624943349,2021-06-29,08:00:00,1,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x736,Safari,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,999960188766601545.1626816843.1626816843,0,232821374.1626816841,2021-07-21,00:00:00,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,NaN,mobile,NaN,Huawei,NaN,360x780,Chrome,Russia,Moscow
1732262,99996598443387715.1626811203.1626811203,0,23282272.1626811203,2021-07-20,23:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,375x667,Safari,Russia,Saint Petersburg
1732263,999966717128502952.1638428330.1638428330,0,232822894.1638428328,2021-12-02,09:58:50,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,NaN,393x873,Chrome,Russia,Nizhny Novgorod
1732264,999988617151873171.1623556243.1623556243,0,232827993.1623556243,2021-06-13,06:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,412x732,Samsung Internet,Russia,Chelyabinsk


посмотрим на состав df_session_target и типы признаков и заметим, что почти все признаки категориальные

In [153]:
df_session_target.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732266 entries, 0 to 1732265
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   session_id                1732266 non-null  object
 1   target                    1732266 non-null  int64 
 2   client_id                 1732266 non-null  object
 3   visit_date                1732266 non-null  object
 4   visit_time                1732266 non-null  object
 5   visit_number              1732266 non-null  int64 
 6   utm_source                1732190 non-null  object
 7   utm_medium                1732266 non-null  object
 8   utm_campaign              1536979 non-null  object
 9   utm_adcontent             1428129 non-null  object
 10  utm_keyword               711514 non-null   object
 11  device_category           1732266 non-null  object
 12  device_os                 718302 non-null   object
 13  device_brand              1385070 non-null

проверяем на пустые значения

In [154]:
df_session_target.eq('').sum()

session_id                  0
target                      0
client_id                   0
visit_date                  0
visit_time                  0
visit_number                0
utm_source                  0
utm_medium                  0
utm_campaign                0
utm_adcontent               0
utm_keyword                 0
device_category             0
device_os                   0
device_brand                0
device_model                0
device_screen_resolution    0
device_browser              0
geo_country                 0
geo_city                    0
dtype: int64

Колличество пропущенных значений по колонкам

In [155]:
df_session_target.isna().sum().sort_values(ascending=False)

device_model                1717204
utm_keyword                 1020752
device_os                   1013964
device_brand                 347196
utm_adcontent                304137
utm_campaign                 195287
utm_source                       76
device_category                   0
geo_country                       0
device_browser                    0
device_screen_resolution          0
session_id                        0
target                            0
utm_medium                        0
visit_number                      0
visit_time                        0
visit_date                        0
client_id                         0
geo_city                          0
dtype: int64

Процент пропущенных значений по колонкам

In [156]:
print('Процент пропущенных значений:')
(df_session_target.isna().sum() / len(df_session_target)).sort_values(ascending=False).apply(lambda x: f'{x:0.2%}')

Процент пропущенных значений:


device_model                99.13%
utm_keyword                 58.93%
device_os                   58.53%
device_brand                20.04%
utm_adcontent               17.56%
utm_campaign                11.27%
utm_source                   0.00%
device_category              0.00%
geo_country                  0.00%
device_browser               0.00%
device_screen_resolution     0.00%
session_id                   0.00%
target                       0.00%
utm_medium                   0.00%
visit_number                 0.00%
visit_time                   0.00%
visit_date                   0.00%
client_id                    0.00%
geo_city                     0.00%
dtype: object

проверим на дубликаты

In [160]:
print('Колличество дубликатов', df_session_target.duplicated().sum())

Колличество дубликатов 0


Удалим неинформативные колонки в которых процент пропусков слишком велик и посмотрим на распределение

In [161]:
df1 = df_session_target.drop(columns=['device_model', 'utm_keyword', 'client_id'], axis=1)
df1.isna().sum().sort_values(ascending=False)

device_os                   1013964
device_brand                 347196
utm_adcontent                304137
utm_campaign                 195287
utm_source                       76
session_id                        0
target                            0
visit_date                        0
visit_time                        0
visit_number                      0
utm_medium                        0
device_category                   0
device_screen_resolution          0
device_browser                    0
geo_country                       0
geo_city                          0
dtype: int64

Заменим пропущенные значения на '(not set)', т.к. это значения встречается в других колонках и оно не определено

In [65]:
# посмотрим сколько неопределенных значений во всех колонках, исходя из значений списка missing_values
def omissions(df):
    missing_values = ['(not set)']
    list_feat = []
    for feat in df:
        list_feat.append([feat, (df[df[feat].isin(missing_values) == True].shape[0])])
    df_omissions = pd.DataFrame(list_feat, columns=['feat', 'omission'])
    return df_omissions[(df_omissions['omission'] != 0)].sort_values('omission', ascending=False, ignore_index=True)

In [66]:
omissions(df1)

,feat,omission
0,geo_city,73297
1,device_brand,16392
2,geo_country,1071
3,utm_medium,405
4,device_os,309
5,device_browser,11


In [67]:
df2 = df1.fillna('(not set)')
df2.isna().sum().sort_values(ascending=False)

session_id                  0
target                      0
visit_date                  0
visit_time                  0
visit_number                0
utm_source                  0
utm_medium                  0
utm_campaign                0
utm_adcontent               0
device_category             0
device_os                   0
device_brand                0
device_screen_resolution    0
device_browser              0
geo_country                 0
geo_city                    0
dtype: int64

In [72]:
collected = gc.collect()
print("«Сборщик мусора: собрано»",
          "%d objects." % collected)

«Сборщик мусора: собрано» 8975 objects.


In [73]:
del df_session_target
del df1

collected = gc.collect()
print("«Сборщик мусора: собрано»",
          "%d objects." % collected)

«Сборщик мусора: собрано» 33 objects.


## *Feature engineering*

### Cоздание новых признаков на основе информации в датафрейме

переведем visit_time в нужный формат и создим новые признаки времени

In [74]:
# переведев в нужный формат
df2['visit_time'] = pd.to_datetime(df2['visit_time'], format='%H:%M:%S')

In [75]:
# час визита
df2['visit_hour'] = df2['visit_time'].dt.hour

будем считать "день" если посщение было в промежутке с 8 до 21 часов, "ночь" остальной промежуток времени суток

In [76]:
# день или ночь визита
df2['visit_day_night'] = df2['visit_time'].dt.hour.apply(lambda x: 'day' if x > 8 and x < 21 else 'night')

переведем visit_date в нужный формат и создим новые признаки даты

In [86]:
# переведем колонку visit_date в datetime64
df2['visit_date'] = pd.to_datetime(df2['visit_date'])

In [87]:
# создадим новый признак день недели визита
df2['dayofweek'] = df2['visit_date'].dt.weekday

In [88]:
# создадим новый признак день визита
df2['visit_day'] = df2['visit_date'].dt.day

In [94]:
# создадим новый признак месяц визита
df2['visit_month'] = df2['visit_date'].dt.month

Согласно указаниям к работе источниками рекламмы в соцсетях из 'utm_source' являются:
 
'QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm'

In [93]:
# создадим переменную
social_media_source = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt',
                        'ISrKoXQCxqqYvAZICvjs', 'IZEXUFLARCUMynmHNBGo',
                        'PlbkrSYoHuZBWfYjYnfw', 'gVRrcxiDQubJiljoTbGm']

создадим новый признак источники рекламмы в соцсетях или нет из 'utm_source'

In [90]:
# создадим новый признак источники рекламмы в соцсетях или нет из 'utm_source'
df2['social_media_source'] = df2['utm_source'].apply(lambda x: 'social' if x in social_media_source else 'no_social')

Согласно указаниям к работе источниками рекламмы в соцсетях из 'utm_medium' являются: 'organic', 'referral', '(none)'

In [91]:
# создадим переменную
organic_traf_medium = ['organic', 'referral', '(none)']

создадим новый признак органический-не органический трафик по колонке utm_medium

In [92]:
# создадим новый признак органический-не органический трафик по колонке utm_medium
df2['traf_medium'] = df2['utm_medium'].apply(lambda x: 'organic' if x in organic_traf_medium else 'no_organic')

создадим признак площадь экрана

In [95]:
df2['device_screen_area'] = df2['device_screen_resolution'].apply(lambda x: int(x.split('x')[0]) * int(x.split('x')[1]))

Создание гео признаков

In [96]:
# заменим Russia на Rusha, чтобы в дальнейшем если нет координат у города найти координаты страны, а Russia определяеся как город в США
df2.loc[:, 'geo_country'] = df2['geo_country'].replace('Russia', 'Rusha')

создадим новый признак Россия или нет

In [98]:
# создадим новый признак Россия или нет
df2['country'] = df2['geo_country'].apply(lambda x: 1.0 if x == 'Rusha' else 0.0)

заменим города с (not set) на страну, чтобы в дальнейшем если город пропущен или координаты не определены, определять координаты страны

In [100]:
# заменим города с (not set) на страну
df2['geo_city'] = df2.apply(lambda x: (x['geo_country'] if x['geo_city'] == '(not set)' else x['geo_city']), axis=1)

создадим словарь городов `city_lat_long` с указанием широты и долготы при помощи `Nominatim`, всего получается 2506 уникальных значений

In [108]:
# список городов
list_city = df2['geo_city'].unique().tolist()
print(len(list_city))

2506


In [ ]:
geocoder = RateLimiter(Nominatim(user_agent='tutorial').geocode, min_delay_seconds=1)
city_lat_long = dict(zip((x for x in list_city), (geocoder('Moscow')[1] if geocoder(x)==None else geocoder(x)[1] for x in list_city)))

In [ ]:
# запишем в словарь
with open(('data/city_lat_long.json'), 'w') as fp:
    json.dump(city_lat_long, fp)

In [109]:
# загрузим словарь с городами и их координатами из data (ранее был создан с использованием geocode)
with open(('data/city_lat_long.json'), 'r') as file:
    city_lat_long = json.load(file)

создадим новый признак широту города

In [110]:
# создадим новый признак широту города
df2.loc[:, 'lat_city'] = df2['geo_city'].apply(lambda x: city_lat_long[x][0] if x != '(not set)' else city_lat_long['Moscow'][0])

создадим новый признак долготу города

In [111]:
# создадим новый признак широту города
df2.loc[:, 'long_city'] = df2['geo_city'].apply(lambda x: city_lat_long[x][1] if x != '(not set)' else city_lat_long['Moscow'][1])

создадим словарь `distance_to_Moscow` с указанием расстояния до Москвы при помощи `distance`, сервис находится в Москве

In [ ]:
distance_to_Moscow = dict(zip(city_lat_long.keys(), (distance.distance(city_lat_long['Moscow'], city_lat_long[x]).km for x in city_lat_long.keys())))

In [ ]:
# запишем в словарь
with open(('data/distance_to_Moscow.json'), 'w') as fp:
    json.dump(distance_to_Moscow, fp)

In [116]:
# загрузим словарь расстояний от Москвы до города
with open(('data/distance_to_Moscow.json'), 'r') as file:
    dist_to_Moscow = json.load(file)

создадим новый признак расстояние от города до Москвы

In [118]:
# создадим новый признак расстояние от города до Москвы
df2.loc[:, 'dist_to_Moscow'] = df2['geo_city'].apply(lambda x: dist_to_Moscow[x] if x != '(not set)' else 0.0)

Удалим не нужные колонки и посмотрим на датасет

In [124]:
# Удаляем ненужные/лишние колонки 
columns_to_drop = [
    'session_id',
    'geo_city',
    'geo_country',
    'device_screen_resolution',
    'utm_medium',
    'utm_source',
    'visit_time',
    'visit_date'
]

In [125]:
df_clean = df2.drop(columns_to_drop, axis=1)
df_clean

,target,visit_number,utm_campaign,utm_adcontent,device_category,device_os,device_brand,device_browser,visit_hour,visit_day_night,dayofweek,social_media_source,traf_medium,visit_day,visit_month,device_screen_area,country,lat_city,long_city,dist_to_Moscow
0,0,1,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,mobile,(not set),Samsung,Chrome,16,day,6,social,no_organic,26,9,358028,1.0,44.560945,38.076683,1231.174609
1,0,1,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,Samsung Internet,21,night,5,no_social,organic,23,10,230400,1.0,51.679931,39.183756,451.327574
2,0,1,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,mobile,Android,Samsung,Chrome,9,day,3,no_social,no_organic,28,10,348552,1.0,44.225383,42.068197,1307.166523
3,0,1,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,mobile,(not set),Realme,Chrome,12,day,5,no_social,no_organic,12,6,280800,1.0,56.637012,104.719221,4004.907350
4,0,1,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,mobile,(not set),Apple,Safari,8,night,1,no_social,organic,29,6,304704,1.0,55.625578,37.606392,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,0,1,FTjNLDyTrXaWYgZymFkV,dUuXlWzvmhDSyclWRhNP,mobile,(not set),Huawei,Chrome,0,night,2,social,no_organic,21,7,280800,1.0,55.625578,37.606392,0.000000
1732262,0,1,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,mobile,(not set),Apple,Safari,23,night,1,no_social,no_organic,20,7,250125,1.0,59.960674,30.158655,654.453737
1732263,0,1,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,mobile,Android,Xiaomi,Chrome,9,day,3,no_social,no_organic,2,12,343089,1.0,56.276929,43.921298,400.960179
1732264,0,1,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,mobile,(not set),Samsung,Samsung Internet,6,night,6,no_social,no_organic,13,6,301584,1.0,55.159841,61.402555,1501.398788


In [126]:
df_clean.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732266 entries, 0 to 1732265
Data columns (total 20 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   target               1732266 non-null  int64  
 1   visit_number         1732266 non-null  int64  
 2   utm_campaign         1732266 non-null  object 
 3   utm_adcontent        1732266 non-null  object 
 4   device_category      1732266 non-null  object 
 5   device_os            1732266 non-null  object 
 6   device_brand         1732266 non-null  object 
 7   device_browser       1732266 non-null  object 
 8   visit_hour           1732266 non-null  int32  
 9   visit_day_night      1732266 non-null  object 
 10  dayofweek            1732266 non-null  int32  
 11  social_media_source  1732266 non-null  object 
 12  traf_medium          1732266 non-null  object 
 13  visit_day            1732266 non-null  int32  
 14  visit_month          1732266 non-null  int32  
 15

## *Преобразование категориальных переменных с помощью OHE и стандартизация данных*

Посмотрим сколько уникальных значений во категориальных колонках

In [132]:
# посмотрим сколько уникальных значений во категориальных колонках
def nunique_feat(df):
    list_feat = []
    categorical = df.select_dtypes(include=['object']).columns
    for feat in categorical:
        list_feat.append([feat, (df[feat].nunique())])
        df_nunique = pd.DataFrame(list_feat, columns=['feat', 'count_uniq'])
    return df_nunique.sort_values('count_uniq', ascending=False, ignore_index=True)

In [133]:
nunique_feat(df_clean)

,feat,count_uniq
0,utm_campaign,407
1,utm_adcontent,281
2,device_brand,200
3,device_browser,55
4,device_os,13
5,device_category,3
6,visit_day_night,2
7,social_media_source,2
8,traf_medium,2


определим категориальные и числовые признаки

In [134]:
# Сохраним в переменную numerical имена всех числовых признаков нашего датасета
numerical = df_clean.select_dtypes(include=['int64', 'int32', 'float64']).columns.drop('target')

# Сохраним в переменную categorical имена всех категориальных признаков нашего датасета
categorical = df_clean.select_dtypes(include=['object']).columns

In [135]:
numerical

Index(['visit_number', 'visit_hour', 'dayofweek', 'visit_day', 'visit_month',
       'device_screen_area', 'country', 'lat_city', 'long_city',
       'dist_to_Moscow'],
      dtype='object')

In [136]:
categorical

Index(['utm_campaign', 'utm_adcontent', 'device_category', 'device_os',
       'device_brand', 'device_browser', 'visit_day_night',
       'social_media_source', 'traf_medium'],
      dtype='object')

*применим min_frequency = 500 для уменьшения размерности датасета*

In [89]:
ohe = OneHotEncoder(handle_unknown='ignore', min_frequency = 500, sparse_output=False)

ohe.fit(df_clean[categorical])

OneHotEncoder(handle_unknown='ignore', min_frequency=500, sparse_output=False)

In [ ]:
df_clean.loc[:, ohe.get_feature_names_out()] = ohe.transform(df_clean[categorical])

стандартизируем данные с помощью StandardScaler

In [92]:
scaler = StandardScaler()
scaler.fit(df_clean[numerical].values)

StandardScaler()

In [ ]:
df_clean.loc[:, scaler.get_feature_names_out([i+'_std' for i in numerical])] = scaler.transform(df_clean[numerical].values)

удалим не переобразованные колонки

In [ ]:
df_clean = df_clean.drop(columns=[*categorical, *numerical], axis=1)

In [97]:
# # запишем в файл csv финальный датасет
# df_clean.to_csv('data/df_ohe_std1.csv', index=False)

## *Modeling*

In [3]:
# # загрузим df_ohe_std
# df_clean = pd.read_csv('data/df_ohe_std1.csv')

разделение данных на треин и тест и объявление моделей


In [7]:
X = df_clean.drop(['target'], axis=1)
y = df_clean['target']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [137]:
logreg = LogisticRegression(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
mlp = MLPClassifier(random_state=42)

логистическая регрессия

In [12]:
logreg.fit(x_train, y_train)
print(roc_auc_score(y_test, logreg.predict_proba(x_test)[:, 1]))

0.6700827562967349


случайный лес

In [ ]:
rf_clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=False, max_features=5, min_samples_split=5,
                       n_jobs=-1, random_state=42)

In [ ]:
rf_pred = rf_clf.predict_proba(x_test)
roc_auc_score(y_test, rf_pred[:, 1])

0.5988990460909874

многослойный персептрон

In [ ]:
mlp.fit(x_train,y_train)

MLPClassifier(activation='logistic', hidden_layer_sizes=(10, 5),
              n_iter_no_change=3, random_state=42)

In [ ]:
mlp_pred = mlp.predict_proba(x_test)
roc_auc_score(y_test, mlp_pred[:, 1])

0.6369697777676482

оценим лучшую модель с помощью кросс-валидации на тренировочной выборке

In [9]:
score_logreg = cross_val_score(logreg, X, y, cv=4, scoring='roc_auc')
print(f'model: {type(logreg).__name__}, acc_mean: {score_logreg.mean():.4f}, acc_std: {score_logreg.std():.4f}')

model: LogisticRegression, acc_mean: 0.6653, acc_std: 0.0027


#### Model tuning с помощью RandomizedSearchCV

In [9]:
# Инициализируем сетку параметров для перебора
param_grid = {
    'solver': ['lbfgs', 'liblinear', 'sag'],
    'class_weight': [None, 'balanced'],
    'max_iter': [100, 200]
}

In [10]:

# Инициализируем базовую модель
logreg2 = LogisticRegression(random_state=22)

# Модель для перебора параметров базовой модели
randomized_search_rf = RandomizedSearchCV(
    estimator=logreg2,
    param_distributions=param_grid,
    n_iter=3,
    scoring='roc_auc',
    verbose=5,
    n_jobs=-1
)

In [11]:
# Выполняем перебор параметров
randomized_search_rf.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


c:\Users\Ser\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
11 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Ser\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Ser\anaconda3\lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "c:\Users\Ser\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1207, in fit
    X, y = self._validate_data(
  File "c:\Users\Ser\anaconda3\lib\site-p

[LibLinear]

RandomizedSearchCV(estimator=LogisticRegression(n_jobs=-1, random_state=22,
                                                verbose=10),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'class_weight': [None, 'balanced'],
                                        'max_iter': [100, 150],
                                        'solver': ['lbfgs', 'liblinear',
                                                   'sag']},
                   scoring='roc_auc', verbose=5)

In [ ]:
# Определим лучшие параметры
best_params = randomized_search_rf.best_params_

{'solver': 'liblinear', 'max_iter': 200, 'class_weight': 'balanced'}


обучим модель на лучших параметрах

In [29]:
logreg2_tuned = LogisticRegression(random_state=42, solver='liblinear', max_iter=200, class_weight='balanced')
logreg2_tuned.fit(x_train, y_train)

preds2 = logreg2_tuned.predict_proba(x_test)[:, 1]
roc_auc_score(y_test, preds2)
print(f'Tuned RF Model Accuracy: {roc_auc_score(y_test, preds2):.5f}')

Tuned RF Model Accuracy: 0.67049


запишем модель в pickle файл

In [24]:
with open('model.pickle', 'wb') as file:
    pickle.dump(logreg2_tuned, file)

# *Results*

В PyCharm соберем pipeline. Результат метрики roc_auc: 0.6874

Условия работы выполнены

![title](data/Pipeline.png)

# API

для тестирования сервиса создадим тестовый файл с полностью заполненными строками

In [ ]:
df_cleaned = df_session_target[~df_session_target.isnull().any(axis=1)]
df_cleaned

,session_id,target,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
153,1000673746057003683.1640596138.1640596138,0,2.329875e+08,2021-12-27,12:08:58,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x780,Chrome,Russia,Krasnodar
193,1000868020311219764.1639112251.1639112251,0,2.330327e+08,2021-12-10,07:57:31,1,ZpYIoDJMcFzVoPFsHGJL,banner,TmThBvoCcwkCZZUWACYq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x780,YaBrowser,Russia,Vladivostok
194,1000868020311219764.1639112419.1639112419,0,2.330327e+08,2021-12-10,08:00:19,2,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x780,YaBrowser,Russia,Vladivostok
409,100190546293664305.1638902376.1638902376,0,2.332743e+07,2021-12-07,21:39:36,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,384x854,Chrome,Russia,Perm
674,1003228169263268578.1637499621.1637499621,0,2.335823e+08,2021-11-21,16:00:21,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x756,Chrome,Russia,Tula
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731698,997376512546207071.1638036831.1638036831,0,2.322198e+08,2021-11-27,21:13:51,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,320x640,Chrome,Russia,Yaroslavl
1731742,997589611644042100.1638513529.1638513529,0,2.322694e+08,2021-12-03,09:38:49,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Vivo,pcvPxfVFaAmhwFmvIeYd,360x780,Chrome,Russia,Lipetsk
1731800,997793317641890856.1632480297.1632480297,0,2.323169e+08,2021-09-24,13:44:57,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,PkybGvWbaqORmxjNunqZ,bGiswZbYCzmVgFSflZDj,mobile,Android,Vivo,pTgAEPipQxDXCjPrJbHo,360x780,Chrome,Russia,Lyubertsy
1731932,998350632605824738.1640085220.1640085220,0,2.324466e+08,2021-12-21,14:13:40,1,ZpYIoDJMcFzVoPFsHGJL,push,sbJRYgVfvcnqKJNDDYIr,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,360x804,Chrome,Russia,Saint Petersburg


In [ ]:
df_test = df_cleaned.sample(n=5)
df_test

,session_id,target,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
580670,3790173640946164764.1638212636.1638212636,1,8.824686e+08,2021-11-29,22:03:56,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Odintsovo
1480801,810034471499726948.1636827235.1636827235,0,1.886008e+08,2021-11-13,21:13:55,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,BmOOOIoWGHEfxEfoUezs,mobile,Android,Vivo,FJApgTrMAGHoxCxQVKws,360x772,Chrome,Russia,Moscow
1631088,881736949078615.1638046298.1638046298,0,2.052952e+05,2021-11-27,23:51:38,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,DpZgNFPlBxxSZpGFMXkk,mobile,Android,Vivo,wsPZygnUifLMgkSEnWLj,360x760,Chrome,Russia,Moscow
858892,5121238582127242988.1639460588.1639460588,0,1.192381e+09,2021-12-14,08:43:08,1,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,393x873,Chrome,Russia,Moscow
433886,3085780128023129986.1640443778.1640443778,0,7.184642e+08,2021-12-25,17:49:38,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),AuMdmADEIoPXiWpTsBEj,412x915,Chrome,Russia,Tver


In [ ]:
test = df_test.to_dict('sample')
test

[{'session_id': '3790173640946164764.1638212636.1638212636',
  'target': 1,
  'client_id': 882468568.1638211,
  'visit_date': '2021-11-29',
  'visit_time': '22:03:56',
  'visit_number': 1,
  'utm_source': 'ZpYIoDJMcFzVoPFsHGJL',
  'utm_medium': 'banner',
  'utm_campaign': 'LEoPHuyFvzoNfnzGgfcd',
  'utm_adcontent': 'vCIpmpaGBnIQhyYNkXqp',
  'utm_keyword': 'puhZPIYqKXeFPaUviSjo',
  'device_category': 'mobile',
  'device_os': 'Android',
  'device_brand': '(not set)',
  'device_model': 'AuMdmADEIoPXiWpTsBEj',
  'device_screen_resolution': '393x873',
  'device_browser': 'Chrome',
  'geo_country': 'Russia',
  'geo_city': 'Odintsovo'},
 {'session_id': '810034471499726948.1636827235.1636827235',
  'target': 0,
  'client_id': 188600847.16368273,
  'visit_date': '2021-11-13',
  'visit_time': '21:13:55',
  'visit_number': 1,
  'utm_source': 'MvfHsxITijuriZxsqZqt',
  'utm_medium': 'cpm',
  'utm_campaign': 'FTjNLDyTrXaWYgZymFkV',
  'utm_adcontent': 'xhoenQgDQsgfEPYNPwKO',
  'utm_keyword': 'BmOOOIoW

запишем данные в файл

In [ ]:
# запишем в файл
with open(('data/test.json'), 'w') as fp:
    json.dump(test, fp)

посмотрим результаты вывода сервиса

![title](data/Postman1.png)

![title](data/Postman2.png)

## *сервис работает, выводит id клиента, id сессии и предсказание*